In [1]:
import os

In [2]:
%pwd

'/Users/wilsvenleong/Downloads/learning-materials/chicken-disease-classification/research'

In [3]:
os.chdir("../")  # Move up to root directory
%pwd

'/Users/wilsvenleong/Downloads/learning-materials/chicken-disease-classification'

## 1. Update `config.yaml`

## 4. Update the Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

## 5. Update the `ConfigurationManager` in `src/config/configuration.py`

In [5]:
from chicken_disease_classification.constants import *
from chicken_disease_classification.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Creates the root directory and returns
        the configuration for data ingestion.

        Returns:
            DataIngestionConfig: Configuration for data ingestion.
        """
        data_ingestion = self.config.data_ingestion

        create_directories([data_ingestion.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(data_ingestion.root_dir),
            source_url=str(data_ingestion.source_url),
            local_data_file=Path(data_ingestion.local_data_file),
            unzip_dir=Path(data_ingestion.unzip_dir),
        )

        return data_ingestion_config

## 6. Update the Components

In [6]:
import os
import zipfile
import urllib.request as request

from chicken_disease_classification import logger
from chicken_disease_classification.utils.common import get_size


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """Downloads the zipped data file if it does not already exist."""
        if not os.path.exists(self.config.local_data_file):
            file_name, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file,
            )
            logger.info(f"{file_name} downloaded with the following info: \n{headers}")
        else:
            logger.info(
                f"File already exists of size: {get_size(self.config.local_data_file)}"
            )

    def extract_zip_file(self):
        """Extracts the contents from zipped file.

        Creates a directory if it does not already exist
        and extracts the contents fo the zipped file into the directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as f:
            f.extractall(unzip_path)

## 7. Update the Pipeline

In [7]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[ 2023-09-22 19:28:04,932 ] 33 common chicken-disease-logger -  INFO - Loaded YAML file successfully from: config/config.yaml
[ 2023-09-22 19:28:04,950 ] 33 common chicken-disease-logger -  INFO - Loaded YAML file successfully from: params.yaml
[ 2023-09-22 19:28:04,951 ] 53 common chicken-disease-logger -  INFO - Created directory at: artifacts
[ 2023-09-22 19:28:04,953 ] 53 common chicken-disease-logger -  INFO - Created directory at: artifacts/data_ingestion
[ 2023-09-22 19:28:11,390 ] 20 2255849282 chicken-disease-logger -  INFO - artifacts/data_ingestion/data.zip downloaded with the following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-Git